#### This juypter notebook shows the process of making the main pandas table for streamlit. Use `Main.py` if you want to run the most up to date version.

## imports :  ]

In [1]:
import json

import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import re

## Prizepicks Table

### Join Prizepicks data into one table

In [2]:
with open("prizepicks_json.json") as f:
    jsonobj = json.load(f)

count = 0
data_attributes = []

for item in jsonobj['data']:
    example = jsonobj['data'][count]
    data_attributes.append(example)
    count += 1

df_attr = pd.json_normalize(data_attributes)
df_attr = df_attr.rename(columns={'relationships.new_player.data.id': 'Player Name'})

count2 = 0
data_included = []

for item in jsonobj['included']:
    example2 = jsonobj['included'][count2]
    data_included.append(example2)
    count2 += 1

df_included = pd.json_normalize(data_included)

df_included = df_included.rename(columns={'id': 'Player Name'})

inner_join = pd.merge(df_attr, df_included, on='Player Name', how='inner')

### Format Prizepicks table

In [3]:
PRIZEPICKS_table = inner_join[['attributes.name', 'attributes.team', 'attributes.league', 'attributes.stat_type', 'attributes.line_score', 'attributes.updated_at']]
PRIZEPICKS_table.columns = ['Player', 'Team', 'Sport', 'Stat Type', 'Line', 'Last Updated']
PRIZEPICKS_table['Last Updated'] = pd.to_datetime(PRIZEPICKS_table['Last Updated'])
PRIZEPICKS_table['Last Updated'] = PRIZEPICKS_table['Last Updated'].dt.strftime('%m/%d/%y %I:%M %p')

This table shows all the current plays on Prizepicks

In [4]:
PRIZEPICKS_table

,Player,Team,Sport,Stat Type,Line,Last Updated
0,Aaron Rai,Aaron Rai,PGA,Fairways Hit,9.0,10/29/22 08:40 PM
1,Aaron Rai,Aaron Rai,PGA,Birdies Or Better,4.5,10/29/22 08:07 PM
2,Aaron Rai,Aaron Rai,PGA,Strokes,68.5,10/29/22 09:27 PM
3,Aaron Rai,Aaron Rai,PGA,Greens In Regulation,12.0,10/29/22 07:50 PM
4,Aaron Rai,Aaron Rai,PGA,Birdies or Better Matchup,0.0,10/29/22 08:24 PM
...,...,...,...,...,...,...
1467,Joshua Cobbs,WYO,CFB,Fantasy Score,6.5,10/29/22 02:01 PM
1468,Zion Bowens,HAW,CFB,Fantasy Score,11.5,10/29/22 10:45 AM
1469,Zion Bowens,HAW,CFB,Receiving Yards,61.5,10/29/22 11:29 AM
1470,Wyatt Wieland,WYO,CFB,Receiving Yards,33.5,10/29/22 10:36 AM


## MLB

#### Clean the MLB data from Draftkings

In [5]:
save_path = 'Draftkings_data\\'
# MLB Hits Allowed
with open(save_path+'draftkings_MLB_HA.txt', 'r') as file:
    HitsAllowed = file.read()
HitsAllowed = HitsAllowed.replace("âˆ’", "-")

HA_Line = re.findall('line">(\d+.\d+)', str(HitsAllowed))

HA_Odds = re.findall('color">(.\d+)', str(HitsAllowed))

HA_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(HitsAllowed))


# MLB Strikeouts
with open(save_path+'draftkings_MLB_SO.txt', 'r') as file:
    Strikeouts = file.read()
Strikeouts = Strikeouts.replace("âˆ’", "-")

SO_Line = re.findall('line">(\d+.\d+)', str(Strikeouts))

SO_Odds = re.findall('color">(.\d+)', str(Strikeouts))

SO_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Strikeouts))

# MLB Runs Scored
with open(save_path+'draftkings_MLB_Runs_Scored.txt', 'r') as file:
    Runs = file.read()
Runs = Runs.replace("âˆ’", "-")

Runs_Line = re.findall('line">(\d+.\d+)', str(Runs))

Runs_Odds = re.findall('color">(.\d+)', str(Runs))

Runs_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Runs))

In [6]:
Upd_Odds = []
Upd_Lines = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in HA_Line:
    Upd_Lines.append(HA_Line[First:Second])
    First += 2
    Second += 2

for i in HA_Odds:
    Upd_Odds.append(HA_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


HA_Odds_DF = pd.DataFrame(Upd_Odds)
HA_Odds_DF = HA_Odds_DF.dropna()

HA_Lines_DF = pd.DataFrame(Upd_Lines)
HA_Lines_DF = HA_Lines_DF.drop([0],axis=1)

In [7]:
Upd_Odds2 = []
Upd_Lines2 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in SO_Line:
    Upd_Lines2.append(SO_Line[First:Second])
    First += 2
    Second += 2

for i in SO_Odds:
    Upd_Odds2.append(SO_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

SO_Odds_DF = pd.DataFrame(Upd_Odds2)
SO_Odds_DF = SO_Odds_DF.dropna()

SO_Lines_DF = pd.DataFrame(Upd_Lines2)
SO_Lines_DF = SO_Lines_DF.dropna()
SO_Lines_DF = SO_Lines_DF.drop([0],axis=1)

In [8]:
Upd_Odds8 = []
Upd_Lines8 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Runs_Line:
    Upd_Lines8.append(Runs_Line[First:Second])
    First += 2
    Second += 2

for i in Runs_Odds:
    Upd_Odds8.append(Runs_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

Runs_Odds_DF = pd.DataFrame(Upd_Odds8)
Runs_Odds_DF = Runs_Odds_DF.dropna()

Runs_Lines_DF = pd.DataFrame(Upd_Lines8)
Runs_Lines_DF = Runs_Lines_DF.dropna()
Runs_Lines_DF = Runs_Lines_DF.drop([0],axis=1)


#### Create dataframes for Draftkings data

In [9]:
HITSALLOWED_DK = pd.DataFrame(list(zip(HA_Players)),
               columns =['Name'])

HITSALLOWED_DK = HITSALLOWED_DK.join(HA_Lines_DF)
HITSALLOWED_DK = HITSALLOWED_DK.join(HA_Odds_DF, lsuffix="fa", rsuffix="werf")

HITSALLOWED_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

HITSALLOWED_DK['O Odds'] = pd.to_numeric(HITSALLOWED_DK['O Odds'])
HITSALLOWED_DK['U Odds'] = pd.to_numeric(HITSALLOWED_DK['U Odds'])
HITSALLOWED_DK['Line'] = pd.to_numeric(HITSALLOWED_DK['Line'])

In [10]:
STRIKEOUTS_DK = pd.DataFrame(list(zip(SO_Players)),
               columns =['Name'])

STRIKEOUTS_DK = STRIKEOUTS_DK.join(SO_Lines_DF)
STRIKEOUTS_DK = STRIKEOUTS_DK.join(SO_Odds_DF, lsuffix="fa", rsuffix="werf")

STRIKEOUTS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

STRIKEOUTS_DK['O Odds'] = pd.to_numeric(STRIKEOUTS_DK['O Odds'])
STRIKEOUTS_DK['U Odds'] = pd.to_numeric(STRIKEOUTS_DK['U Odds'])
STRIKEOUTS_DK['Line'] = pd.to_numeric(STRIKEOUTS_DK['Line'])

In [11]:
RUNS_DK = pd.DataFrame(list(zip(Runs_Players)),
               columns =['Name'])

RUNS_DK = RUNS_DK.join(Runs_Lines_DF)
RUNS_DK = RUNS_DK.join(Runs_Odds_DF, lsuffix="fa", rsuffix="werf")

RUNS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RUNS_DK['O Odds'] = pd.to_numeric(RUNS_DK['O Odds'])
RUNS_DK['U Odds'] = pd.to_numeric(RUNS_DK['U Odds'])
RUNS_DK['Line'] = pd.to_numeric(RUNS_DK['Line'])


#### Format dataframes with probability formula

In [12]:
STRIKEOUTS_DK['O Prob'] = (STRIKEOUTS_DK['O Odds'].abs()/(STRIKEOUTS_DK['O Odds'].abs()+100)*100).round(1).where(STRIKEOUTS_DK['O Odds'] < 0, 100/(STRIKEOUTS_DK['O Odds']+100)*100).round(1)
STRIKEOUTS_DK['U Prob'] = (STRIKEOUTS_DK['U Odds'].abs()/(STRIKEOUTS_DK['U Odds'].abs()+100)*100).round(1).where(STRIKEOUTS_DK['U Odds'] < 0, 100/(STRIKEOUTS_DK['U Odds']+100)*100).round(1)
STRIKEOUTS_DK['Favor'] = np.where(STRIKEOUTS_DK['O Prob']> STRIKEOUTS_DK['U Prob'], "Over", "Under")

STRIKEOUTS_DK = STRIKEOUTS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
STRIKEOUTS_DK.head()

,Player,Line,O Prob,U Prob,Favor


In [13]:
HITSALLOWED_DK['O Prob'] = (HITSALLOWED_DK['O Odds'].abs()/(HITSALLOWED_DK['O Odds'].abs()+100)*100).round(1).where(HITSALLOWED_DK['O Odds'] < 0, 100/(HITSALLOWED_DK['O Odds']+100)*100).round(1)
HITSALLOWED_DK['U Prob'] = (HITSALLOWED_DK['U Odds'].abs()/(HITSALLOWED_DK['U Odds'].abs()+100)*100).round(1).where(HITSALLOWED_DK['U Odds'] < 0, 100/(HITSALLOWED_DK['U Odds']+100)*100).round(1)
HITSALLOWED_DK['Favor'] = np.where(HITSALLOWED_DK['O Prob']> HITSALLOWED_DK['U Prob'], "Over", "Under")

HITSALLOWED_DK = HITSALLOWED_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
HITSALLOWED_DK.head()

,Player,Line,O Prob,U Prob,Favor


In [14]:
RUNS_DK['O Prob'] = (RUNS_DK['O Odds'].abs()/(RUNS_DK['O Odds'].abs()+100)*100).round(1).where(RUNS_DK['O Odds'] < 0, 100/(RUNS_DK['O Odds']+100)*100).round(1)
RUNS_DK['U Prob'] = (RUNS_DK['U Odds'].abs()/(RUNS_DK['U Odds'].abs()+100)*100).round(1).where(RUNS_DK['U Odds'] < 0, 100/(RUNS_DK['U Odds']+100)*100).round(1)
RUNS_DK['Favor'] = np.where(RUNS_DK['O Prob']> RUNS_DK['U Prob'], "Over", "Under")

RUNS_DK = RUNS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RUNS_DK.head()

,Player,Line,O Prob,U Prob,Favor


#### Match lines from Draftkings to Prizepicks

In [15]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


STRIKEOUTS_TABLE = fuzzy_merge(PRIZEPICKS_table, STRIKEOUTS_DK, 'Player', 'Player', threshold=95)

STRIKEOUTS_TABLE = STRIKEOUTS_TABLE[STRIKEOUTS_TABLE['Stat Type'].str.contains('Strikeout')]
STRIKEOUTS_TABLE = STRIKEOUTS_TABLE[STRIKEOUTS_TABLE['Sport'].str.endswith('MLB')]
STRIKEOUTS_TABLE['Player'] = STRIKEOUTS_TABLE['matches']

STRIKEOUTS_TABLE = STRIKEOUTS_TABLE.merge(STRIKEOUTS_DK, on='Player')

STRIKEOUTS_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [16]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


HITSALLOWED_TABLE = fuzzy_merge(PRIZEPICKS_table, HITSALLOWED_DK, 'Player', 'Player', threshold=95)

HITSALLOWED_TABLE = HITSALLOWED_TABLE[HITSALLOWED_TABLE['Stat Type'].str.contains('Hits Allowed')]
HITSALLOWED_TABLE = HITSALLOWED_TABLE[HITSALLOWED_TABLE['Sport'].str.endswith('MLB')]
HITSALLOWED_TABLE['Player'] = HITSALLOWED_TABLE['matches']

HITSALLOWED_TABLE = HITSALLOWED_TABLE.merge(HITSALLOWED_DK, on='Player')

HITSALLOWED_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [17]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RUNS_TABLE = fuzzy_merge(PRIZEPICKS_table, RUNS_DK, 'Player', 'Player', threshold=95)

RUNS_TABLE = RUNS_TABLE[RUNS_TABLE['Stat Type'].str.endswith('Runs')]
RUNS_TABLE = RUNS_TABLE[RUNS_TABLE['Sport'].str.endswith('MLB')]
RUNS_TABLE['Player'] = RUNS_TABLE['matches']

RUNS_TABLE = RUNS_TABLE.merge(RUNS_DK, on='Player')

RUNS_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


## NFL

#### Clean the NFL data from Draftkings

In [18]:
# NFL Passing Yards
with open(save_path+'draftkings_NFL_PassYds.txt', 'r') as file:
    PassYds = file.read()
PassYds = PassYds.replace("âˆ’", "-")

PassYds_Line = re.findall('line">(\d+.\d+)', str(PassYds))

PassYds_Odds = re.findall('color">(.\d+)', str(PassYds))

PassYds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(PassYds))


# NFL Passing Touchdowns
with open(save_path+'draftkings_NFL_PassTds.txt', 'r') as file:
    PassTds = file.read()
PassTds = PassTds.replace("âˆ’", "-")

PassTds_Line = re.findall('line">(\d+.\d+)', str(PassTds))

PassTds_Odds = re.findall('color">(.\d+)', str(PassTds))

PassTds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(PassTds))

# NFL Rush Yards
with open(save_path+'draftkings_NFL_RushYds.txt', 'r') as file:
    RushYds = file.read()
RushYds = RushYds.replace("âˆ’", "-")

RushYds_Line = re.findall('line">(\d+.\d+)', str(RushYds))

RushYds_Odds = re.findall('color">(.\d+)', str(RushYds))

RushYds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(RushYds))

# NFL Rush Attempts
with open(save_path+'draftkings_NFL_RushAtt.txt', 'r') as file:
    RushAtt = file.read()
RushAtt = RushAtt.replace("âˆ’", "-")

RushAtt_Line = re.findall('line">(\d+.\d+)', str(RushAtt))

RushAtt_Odds = re.findall('color">(.\d+)', str(RushAtt))

RushAtt_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(RushAtt))

# NFL Receptions
with open(save_path+'draftkings_NFL_Rec.txt', 'r') as file:
    Rec = file.read()
Rec = Rec.replace("âˆ’", "-")

Rec_Line = re.findall('line">(\d+.\d+)', str(Rec))

Rec_Odds = re.findall('color">(.\d+)', str(Rec))

Rec_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Rec))

# NFL Receiving Yards
with open(save_path+'draftkings_NFL_RecYds.txt', 'r') as file:
    RECYDS = file.read()
RECYDS = RECYDS.replace("âˆ’", "-")

RECYDS_Line = re.findall('line">(\d+.\d+)', str(RECYDS))

RECYDS_Odds = re.findall('color">(.\d+)', str(RECYDS))

RECYDS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(RECYDS))

# NFL Pass Attempts
with open(save_path+'draftkings_NFL_PassAtt.txt', 'r') as file:
    PassAtt = file.read()
PassAtt = PassAtt.replace("âˆ’", "-")

PassAtt_Line = re.findall('line">(\d+.\d+)', str(PassAtt))

PassAtt_Odds = re.findall('color">(.\d+)', str(PassAtt))

PassAtt_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(PassAtt))

In [19]:
Upd_Odds3 = []
Upd_Lines3 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in PassYds_Line:
    Upd_Lines3.append(PassYds_Line[First:Second])
    First += 2
    Second += 2

for i in PassYds_Odds:
    Upd_Odds3.append(PassYds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


PassYds_Odds_DF = pd.DataFrame(Upd_Odds3)
PassYds_Odds_DF = PassYds_Odds_DF.dropna()

PassYds_Lines_DF = pd.DataFrame(Upd_Lines3)
PassYds_Lines_DF = PassYds_Lines_DF.dropna()
PassYds_Lines_DF = PassYds_Lines_DF.drop([0],axis=1)

In [20]:
Upd_Odds4 = []
Upd_Lines4 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in PassTds_Line:
    Upd_Lines4.append(PassTds_Line[First:Second])
    First += 2
    Second += 2

for i in PassTds_Odds:
    Upd_Odds4.append(PassTds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

PassTds_Odds_DF = pd.DataFrame(Upd_Odds4)
PassTds_Odds_DF = PassTds_Odds_DF.dropna()

PassTds_Lines_DF = pd.DataFrame(Upd_Lines4)
PassTds_Lines_DF = PassTds_Lines_DF.dropna()
PassTds_Lines_DF = PassTds_Lines_DF.drop([0],axis=1)


In [21]:
Upd_Odds5 = []
Upd_Lines5 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in RushYds_Line:
    Upd_Lines5.append(RushYds_Line[First:Second])
    First += 2
    Second += 2

for i in RushYds_Odds:
    Upd_Odds5.append(RushYds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

RushYds_Odds_DF = pd.DataFrame(Upd_Odds5)
RushYds_Odds_DF = RushYds_Odds_DF.dropna()

RushYds_Lines_DF = pd.DataFrame(Upd_Lines5)
RushYds_Lines_DF = RushYds_Lines_DF.dropna()
RushYds_Lines_DF = RushYds_Lines_DF.drop([0],axis=1)


In [22]:
Upd_Odds9 = []
Upd_Lines9 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in RushAtt_Line:
    Upd_Lines9.append(RushAtt_Line[First:Second])
    First += 2
    Second += 2

for i in RushAtt_Odds:
    Upd_Odds9.append(RushAtt_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

RushAtt_Odds_DF = pd.DataFrame(Upd_Odds9)
RushAtt_Odds_DF = RushAtt_Odds_DF.dropna()

RushAtt_Lines_DF = pd.DataFrame(Upd_Lines9)
RushAtt_Lines_DF = RushAtt_Lines_DF.dropna()
RushAtt_Lines_DF = RushAtt_Lines_DF.drop([0],axis=1)


In [23]:
Upd_Odds12 = []
Upd_Lines12 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Rec_Line:
    Upd_Lines12.append(Rec_Line[First:Second])
    First += 2
    Second += 2

for i in Rec_Odds:
    Upd_Odds12.append(Rec_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

Rec_Odds_DF = pd.DataFrame(Upd_Odds12)
Rec_Odds_DF = Rec_Odds_DF.dropna()

Rec_Lines_DF = pd.DataFrame(Upd_Lines12)
Rec_Lines_DF = Rec_Lines_DF.dropna()
Rec_Lines_DF = Rec_Lines_DF.drop([0],axis=1)


In [24]:
Upd_Odds21 = []
Upd_Lines21 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in RECYDS_Line:
    Upd_Lines21.append(RECYDS_Line[First:Second])
    First += 2
    Second += 2

for i in RECYDS_Odds:
    Upd_Odds21.append(RECYDS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

RECYDS_Odds_DF = pd.DataFrame(Upd_Odds21)
RECYDS_Odds_DF = RECYDS_Odds_DF.dropna()

RECYDS_Lines_DF = pd.DataFrame(Upd_Lines21)
RECYDS_Lines_DF = RECYDS_Lines_DF.dropna()
RECYDS_Lines_DF = RECYDS_Lines_DF.drop([0],axis=1)


In [25]:
Upd_Odds25 = []
Upd_Lines25 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in PassAtt_Line:
    Upd_Lines25.append(PassAtt_Line[First:Second])
    First += 2
    Second += 2

for i in PassAtt_Odds:
    Upd_Odds25.append(PassAtt_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

PassAtt_Odds_DF = pd.DataFrame(Upd_Odds25)
PassAtt_Odds_DF = PassAtt_Odds_DF.dropna()

PassAtt_Lines_DF = pd.DataFrame(Upd_Lines25)
PassAtt_Lines_DF = PassAtt_Lines_DF.dropna()
PassAtt_Lines_DF = PassAtt_Lines_DF.drop([0],axis=1)


#### Create dataframes for Draftkings data

In [26]:
PASSYDS_DK = pd.DataFrame(list(zip(PassYds_Players)),
               columns =['Name'])

PASSYDS_DK = PASSYDS_DK.join(PassYds_Lines_DF)
PASSYDS_DK = PASSYDS_DK.join(PassYds_Odds_DF, lsuffix="fa", rsuffix="werf")

PASSYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

PASSYDS_DK[['O Odds', 'Line', 'U Odds']] = PASSYDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [27]:
PASSTDS_DK = pd.DataFrame(list(zip(PassTds_Players)),
               columns =['Name'])

PASSTDS_DK = PASSTDS_DK.join(PassTds_Lines_DF)
PASSTDS_DK = PASSTDS_DK.join(PassTds_Odds_DF, lsuffix="fa", rsuffix="werf")

PASSTDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

PASSTDS_DK[['O Odds', 'Line', 'U Odds']] = PASSTDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)


In [28]:
RUSHYDS_DK = pd.DataFrame(list(zip(RushYds_Players)),
               columns =['Name'])

RUSHYDS_DK = RUSHYDS_DK.join(RushYds_Lines_DF)
RUSHYDS_DK = RUSHYDS_DK.join(RushYds_Odds_DF, lsuffix="fa", rsuffix="werf")

RUSHYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RUSHYDS_DK[['O Odds', 'Line', 'U Odds']] = RUSHYDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [29]:
RUSHATT_DK = pd.DataFrame(list(zip(RushAtt_Players)),
               columns =['Name'])

RUSHATT_DK = RUSHATT_DK.join(RushAtt_Lines_DF)
RUSHATT_DK = RUSHATT_DK.join(RushAtt_Odds_DF, lsuffix="fa", rsuffix="werf")

RUSHATT_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RUSHATT_DK[['O Odds', 'Line', 'U Odds']] = RUSHATT_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [30]:
REC_DK = pd.DataFrame(list(zip(Rec_Players)),
               columns =['Name'])

REC_DK = REC_DK.join(Rec_Lines_DF)
REC_DK = REC_DK.join(Rec_Odds_DF, lsuffix="fa", rsuffix="werf")

REC_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

REC_DK[['O Odds', 'Line', 'U Odds']] = REC_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [31]:
RECYDS_DK = pd.DataFrame(list(zip(RECYDS_Players)),
               columns =['Name'])

RECYDS_DK = RECYDS_DK.join(RECYDS_Lines_DF)
RECYDS_DK = RECYDS_DK.join(RECYDS_Odds_DF, lsuffix="fa", rsuffix="werf")

RECYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

RECYDS_DK[['O Odds', 'Line', 'U Odds']] = RECYDS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [32]:
PassAtt_DK = pd.DataFrame(list(zip(PassAtt_Players)),
               columns =['Name'])

PassAtt_DK = PassAtt_DK.join(PassAtt_Lines_DF)
PassAtt_DK = PassAtt_DK.join(PassAtt_Odds_DF, lsuffix="fa", rsuffix="werf")

PassAtt_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

PassAtt_DK[['O Odds', 'Line', 'U Odds']] = PassAtt_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

#### Format dataframes with probability formula

In [33]:
PASSYDS_DK['O Prob'] = (PASSYDS_DK['O Odds'].abs()/(PASSYDS_DK['O Odds'].abs()+100)*100).round(1).where(PASSYDS_DK['O Odds'] < 0, 100/(PASSYDS_DK['O Odds']+100)*100).round(1)
PASSYDS_DK['U Prob'] = (PASSYDS_DK['U Odds'].abs()/(PASSYDS_DK['U Odds'].abs()+100)*100).round(1).where(PASSYDS_DK['U Odds'] < 0, 100/(PASSYDS_DK['U Odds']+100)*100).round(1)
PASSYDS_DK['Favor'] = np.where(PASSYDS_DK['O Prob']> PASSYDS_DK['U Prob'], "Over", "Under")

PASSYDS_DK = PASSYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
PASSYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Russell Wilson,226.5,53.5,53.5,Under
1,Trevor Lawrence,218.5,53.5,53.5,Under
2,Jalen Hurts,227.5,54.5,52.4,Over
3,Kenny Pickett,218.5,53.5,53.5,Under
4,Mac Jones,196.5,53.5,53.5,Under


In [34]:
PASSTDS_DK['O Prob'] = (PASSTDS_DK['O Odds'].abs()/(PASSTDS_DK['O Odds'].abs()+100)*100).round(1).where(PASSTDS_DK['O Odds'] < 0, 100/(PASSTDS_DK['O Odds']+100)*100).round(1)
PASSTDS_DK['U Prob'] = (PASSTDS_DK['U Odds'].abs()/(PASSTDS_DK['U Odds'].abs()+100)*100).round(1).where(PASSTDS_DK['U Odds'] < 0, 100/(PASSTDS_DK['U Odds']+100)*100).round(1)
PASSTDS_DK['Favor'] = np.where(PASSTDS_DK['O Prob']> PASSTDS_DK['U Prob'], "Over", "Under")

PASSTDS_DK = PASSTDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
PASSTDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Russell Wilson,1.5,42.6,64.3,Under
1,Trevor Lawrence,1.5,40.0,67.2,Under
2,Jalen Hurts,1.5,48.8,57.4,Under
3,Kenny Pickett,1.5,33.9,73.3,Under
4,Mac Jones,1.5,40.8,66.7,Under


In [35]:
RUSHYDS_DK['O Prob'] = (RUSHYDS_DK['O Odds'].abs()/(RUSHYDS_DK['O Odds'].abs()+100)*100).round(1).where(RUSHYDS_DK['O Odds'] < 0, 100/(RUSHYDS_DK['O Odds']+100)*100).round(1)
RUSHYDS_DK['U Prob'] = (RUSHYDS_DK['U Odds'].abs()/(RUSHYDS_DK['U Odds'].abs()+100)*100).round(1).where(RUSHYDS_DK['U Odds'] < 0, 100/(RUSHYDS_DK['U Odds']+100)*100).round(1)
RUSHYDS_DK['Favor'] = np.where(RUSHYDS_DK['O Prob']> RUSHYDS_DK['U Prob'], "Over", "Under")

RUSHYDS_DK = RUSHYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RUSHYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Latavius Murray,32.5,53.5,53.5,Under
1,Melvin Gordon I,39.5,52.4,54.5,Under
2,Russell Wilson,12.5,48.8,56.5,Under
3,Travis Etienne,71.5,45.5,60.0,Under
4,Trevor Lawrence,12.5,53.5,53.5,Under


In [36]:
RUSHATT_DK['O Prob'] = (RUSHATT_DK['O Odds'].abs()/(RUSHATT_DK['O Odds'].abs()+100)*100).round(1).where(RUSHATT_DK['O Odds'] < 0, 100/(RUSHATT_DK['O Odds']+100)*100).round(1)
RUSHATT_DK['U Prob'] = (RUSHATT_DK['U Odds'].abs()/(RUSHATT_DK['U Odds'].abs()+100)*100).round(1).where(RUSHATT_DK['U Odds'] < 0, 100/(RUSHATT_DK['U Odds']+100)*100).round(1)
RUSHATT_DK['Favor'] = np.where(RUSHATT_DK['O Prob']> RUSHATT_DK['U Prob'], "Over", "Under")

RUSHATT_DK = RUSHATT_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RUSHATT_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Latavius Murray,7.5,59.2,47.6,Over
1,Melvin Gordon I,10.5,54.5,51.2,Over
2,Travis Etienne,15.5,56.5,50.0,Over
3,Jalen Hurts,9.5,59.2,47.6,Over
4,Miles Sanders,15.5,48.8,57.4,Under


In [37]:
REC_DK['O Prob'] = (REC_DK['O Odds'].abs()/(REC_DK['O Odds'].abs()+100)*100).round(1).where(REC_DK['O Odds'] < 0, 100/(REC_DK['O Odds']+100)*100).round(1)
REC_DK['U Prob'] = (REC_DK['U Odds'].abs()/(REC_DK['U Odds'].abs()+100)*100).round(1).where(REC_DK['U Odds'] < 0, 100/(REC_DK['U Odds']+100)*100).round(1)
REC_DK['Favor'] = np.where(REC_DK['O Prob']> REC_DK['U Prob'], "Over", "Under")

REC_DK = REC_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
REC_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Christian Kirk,4.5,54.5,52.4,Over
1,Courtland Sutton,3.5,63.6,43.5,Over
2,Evan Engram,3.5,45.5,61.5,Under
3,Greg Dulcich,3.5,46.5,60.0,Under
4,Jerry Jeudy,3.5,63.6,43.5,Over


In [38]:
RECYDS_DK['O Prob'] = (RECYDS_DK['O Odds'].abs()/(RECYDS_DK['O Odds'].abs()+100)*100).round(1).where(RECYDS_DK['O Odds'] < 0, 100/(RECYDS_DK['O Odds']+100)*100).round(1)
RECYDS_DK['U Prob'] = (RECYDS_DK['U Odds'].abs()/(RECYDS_DK['U Odds'].abs()+100)*100).round(1).where(RECYDS_DK['U Odds'] < 0, 100/(RECYDS_DK['U Odds']+100)*100).round(1)
RECYDS_DK['Favor'] = np.where(RECYDS_DK['O Prob']> RECYDS_DK['U Prob'], "Over", "Under")

RECYDS_DK = RECYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
RECYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Christian Kirk,50.5,53.5,53.5,Under
1,Courtland Sutton,55.5,52.4,54.5,Under
2,Evan Engram,30.5,53.5,53.5,Under
3,Greg Dulcich,33.5,53.5,53.5,Under
4,Jerry Jeudy,47.5,53.5,53.5,Under


In [39]:
PassAtt_DK['O Prob'] = (PassAtt_DK['O Odds'].abs()/(PassAtt_DK['O Odds'].abs()+100)*100).round(1).where(PassAtt_DK['O Odds'] < 0, 100/(PassAtt_DK['O Odds']+100)*100).round(1)
PassAtt_DK['U Prob'] = (PassAtt_DK['U Odds'].abs()/(PassAtt_DK['U Odds'].abs()+100)*100).round(1).where(PassAtt_DK['U Odds'] < 0, 100/(PassAtt_DK['U Odds']+100)*100).round(1)
PassAtt_DK['Favor'] = np.where(PassAtt_DK['O Prob']> PassAtt_DK['U Prob'], "Over", "Under")

PassAtt_DK = PassAtt_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
PassAtt_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Russell Wilson,31.5,52.4,54.5,Under
1,Trevor Lawrence,32.5,54.5,52.4,Over
2,Jalen Hurts,30.5,52.4,54.5,Under
3,Kenny Pickett,35.5,63.0,44.4,Over
4,Zach Wilson,27.5,54.5,52.4,Over


#### Match lines from Draftkings to Prizepicks

In [40]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


PASSYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, PASSYDS_DK, 'Player', 'Player', threshold=95)

PASSYDS_TABLE = PASSYDS_TABLE[PASSYDS_TABLE['Stat Type'].str.contains('Pass Yards')]
PASSYDS_TABLE = PASSYDS_TABLE[PASSYDS_TABLE['Sport'].str.endswith('NFL')]
PASSYDS_TABLE['Player'] = PASSYDS_TABLE['matches']

PASSYDS_TABLE = PASSYDS_TABLE.merge(PASSYDS_DK, on='Player')

PASSYDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Russell Wilson,DEN,NFL,Pass Yards,220.5,10/28/22 12:30 PM,Russell Wilson,226.5,53.5,53.5,Under
1,Trevor Lawrence,JAC,NFL,Pass Yards,215.5,10/26/22 06:57 PM,Trevor Lawrence,218.5,53.5,53.5,Under
2,Kyler Murray,ARI,NFL,Pass Yards,248.5,10/28/22 04:37 PM,Kyler Murray,249.5,53.5,53.5,Under
3,Kirk Cousins,MIN,NFL,Pass Yards,265.5,10/25/22 02:22 PM,Kirk Cousins,272.5,53.5,53.5,Under
4,Tua Tagovailoa,MIA,NFL,Pass Yards,270.5,10/27/22 11:43 AM,Tua Tagovailoa,273.5,54.5,52.4,Over


In [41]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


PASSTDS_TABLE = fuzzy_merge(PRIZEPICKS_table, PASSTDS_DK, 'Player', 'Player', threshold=95)

PASSTDS_TABLE = PASSTDS_TABLE[PASSTDS_TABLE['Stat Type'].str.contains('Pass TDs')]
PASSTDS_TABLE = PASSTDS_TABLE[PASSTDS_TABLE['Sport'].str.endswith('NFL')]
PASSTDS_TABLE['Player'] = PASSTDS_TABLE['matches']

PASSTDS_TABLE = PASSTDS_TABLE.merge(PASSTDS_DK, on='Player')

PASSTDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Kyler Murray,ARI,NFL,Pass TDs,1.5,10/28/22 05:02 PM,Kyler Murray,1.5,48.8,58.3,Under
1,Dak Prescott,DAL,NFL,Pass TDs,1.5,10/27/22 01:34 AM,Dak Prescott,1.5,57.4,48.8,Over
2,Derek Carr,LV,NFL,Pass TDs,1.5,10/27/22 01:46 AM,Derek Carr,1.5,57.4,50.0,Over
3,Andy Dalton,NO,NFL,Pass TDs,1.5,10/27/22 01:34 AM,Andy Dalton,1.5,48.8,57.4,Under
4,Matthew Stafford,LA,NFL,Pass TDs,1.5,10/28/22 07:31 PM,Matthew Stafford,1.5,57.4,50.0,Over


In [42]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RUSHATT_TABLE = fuzzy_merge(PRIZEPICKS_table, RUSHATT_DK, 'Player', 'Player', threshold=95)

RUSHATT_TABLE = RUSHATT_TABLE[RUSHATT_TABLE['Stat Type'].str.contains('Rush Att')]
RUSHATT_TABLE = RUSHATT_TABLE[RUSHATT_TABLE['Sport'].str.endswith('NFL')]
RUSHATT_TABLE['Player'] = RUSHATT_TABLE['matches']

RUSHATT_TABLE = RUSHATT_TABLE.merge(RUSHATT_DK, on='Player')

RUSHATT_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Melvin Gordon I,DEN,NFL,Rush Attempts,10.5,10/28/22 01:10 PM,Melvin Gordon I,10.5,54.5,51.2,Over
1,Latavius Murray,DEN,NFL,Rush Attempts,8.5,10/28/22 12:35 PM,Latavius Murray,7.5,59.2,47.6,Over
2,Raheem Mostert,MIA,NFL,Rush Attempts,15.5,10/28/22 03:50 PM,Raheem Mostert,15.5,54.5,52.4,Over
3,Josh Jacobs,LV,NFL,Rush Attempts,19.5,10/28/22 05:36 PM,Josh Jacobs,19.5,54.5,52.4,Over
4,Alvin Kamara,NO,NFL,Rush Attempts,15.0,10/28/22 05:36 PM,Alvin Kamara,14.5,57.4,48.8,Over


In [43]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RUSHYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, RUSHYDS_DK, 'Player', 'Player', threshold=95)

RUSHYDS_TABLE = RUSHYDS_TABLE[RUSHYDS_TABLE['Stat Type'].str.contains('Rush Yards')]
RUSHYDS_TABLE = RUSHYDS_TABLE[RUSHYDS_TABLE['Sport'].str.endswith('NFL')]
RUSHYDS_TABLE['Player'] = RUSHYDS_TABLE['matches']

RUSHYDS_TABLE = RUSHYDS_TABLE.merge(RUSHYDS_DK, on='Player')

RUSHYDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Trevor Lawrence,JAC,NFL,Rush Yards,11.5,10/27/22 01:05 PM,Trevor Lawrence,12.5,53.5,53.5,Under
1,Melvin Gordon I,DEN,NFL,Rush Yards,39.5,10/27/22 01:12 PM,Melvin Gordon I,39.5,52.4,54.5,Under
2,Travis Etienne,JAC,NFL,Rush Yards,69.5,10/27/22 12:43 PM,Travis Etienne,71.5,45.5,60.0,Under
3,Latavius Murray,DEN,NFL,Rush Yards,37.5,10/27/22 01:07 PM,Latavius Murray,32.5,53.5,53.5,Under
4,Kyler Murray,ARI,NFL,Rush Yards,32.5,10/28/22 05:04 PM,Kyler Murray,32.5,53.5,53.5,Under


In [44]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


REC_TABLE = fuzzy_merge(PRIZEPICKS_table, REC_DK, 'Player', 'Player', threshold=95)

REC_TABLE = REC_TABLE[REC_TABLE['Stat Type'].str.contains('Receptions')]
REC_TABLE = REC_TABLE[REC_TABLE['Sport'].str.endswith('NFL')]
REC_TABLE['Player'] = REC_TABLE['matches']

REC_TABLE = REC_TABLE.merge(REC_DK, on='Player')

REC_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Melvin Gordon I,DEN,NFL,Receptions,2.0,10/28/22 12:34 PM,Melvin Gordon I,1.5,64.9,42.6,Over
1,Travis Etienne,JAC,NFL,Receptions,3.0,10/28/22 12:34 PM,Travis Etienne,3.5,42.6,64.3,Under
2,Christian Kirk,JAC,NFL,Receptions,5.0,10/28/22 12:41 PM,Christian Kirk,4.5,54.5,52.4,Over
3,Courtland Sutton,DEN,NFL,Receptions,4.0,10/28/22 12:33 PM,Courtland Sutton,3.5,63.6,43.5,Over
4,Zay Jones,JAC,NFL,Receptions,4.0,10/28/22 12:36 PM,Zay Jones,3.5,62.3,44.4,Over


In [45]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


RECYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, RECYDS_DK, 'Player', 'Player', threshold=95)

RECYDS_TABLE = RECYDS_TABLE[RECYDS_TABLE['Stat Type'].str.contains('Receiving Yard')]
RECYDS_TABLE = RECYDS_TABLE[RECYDS_TABLE['Sport'].str.endswith('NFL')]
RECYDS_TABLE['Player'] = RECYDS_TABLE['matches']

RECYDS_TABLE = RECYDS_TABLE.merge(RECYDS_DK, on='Player')

RECYDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Melvin Gordon I,DEN,NFL,Receiving Yards,10.5,10/28/22 12:20 PM,Melvin Gordon I,12.5,52.4,54.5,Under
1,Travis Etienne,JAC,NFL,Receiving Yards,20.5,10/27/22 01:13 PM,Travis Etienne,19.5,53.5,53.5,Under
2,Christian Kirk,JAC,NFL,Receiving Yards,50.5,10/27/22 12:17 AM,Christian Kirk,50.5,53.5,53.5,Under
3,Courtland Sutton,DEN,NFL,Receiving Yards,54.5,10/27/22 01:38 PM,Courtland Sutton,55.5,52.4,54.5,Under
4,Zay Jones,JAC,NFL,Receiving Yards,42.5,10/27/22 01:20 PM,Zay Jones,42.5,52.4,54.5,Under


In [46]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


PassAtt_TABLE = fuzzy_merge(PRIZEPICKS_table, PassAtt_DK, 'Player', 'Player', threshold=95)

PassAtt_TABLE = PassAtt_TABLE[PassAtt_TABLE['Stat Type'].str.contains('Pass Attempts')]
PassAtt_TABLE = PassAtt_TABLE[PassAtt_TABLE['Sport'].str.endswith('NFL')]
PassAtt_TABLE['Player'] = PassAtt_TABLE['matches']

PassAtt_TABLE = PassAtt_TABLE.merge(PassAtt_DK, on='Player')

PassAtt_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Trevor Lawrence,JAC,NFL,Pass Attempts,32.5,10/28/22 12:31 PM,Trevor Lawrence,32.5,54.5,52.4,Over
1,Tua Tagovailoa,MIA,NFL,Pass Attempts,34.5,10/28/22 03:45 PM,Tua Tagovailoa,35.5,51.2,55.6,Under
2,Jared Goff,DET,NFL,Pass Attempts,34.5,10/28/22 03:42 PM,Jared Goff,34.5,54.5,52.4,Over
3,Dak Prescott,DAL,NFL,Pass Attempts,31.5,10/28/22 01:10 PM,Dak Prescott,31.5,53.5,53.5,Under
4,Andy Dalton,NO,NFL,Pass Attempts,32.5,10/28/22 05:38 PM,Andy Dalton,32.5,54.5,52.4,Over


## NBA

#### Clean the NBA data from Draftkings

In [47]:
# NBA Points
with open(save_path+'draftkings_NBA_PTS.txt', 'r') as file:
    NBA_PTS = file.read()
NBA_PTS = NBA_PTS.replace("âˆ’", "-")

NBA_PTS_Line = re.findall('line">(\d+.\d+)', str(NBA_PTS))

NBA_PTS_Odds = re.findall('color">(.\d+)', str(NBA_PTS))

NBA_PTS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(NBA_PTS))

# NBA Rebounds
with open(save_path+'draftkings_NBA_Rebounds.txt', 'r') as file:
    Rebounds = file.read()
Rebounds = Rebounds.replace("âˆ’", "-")

Rebounds_Line = re.findall('line">(\d+.\d+)', str(Rebounds))

Rebounds_Odds = re.findall('color">(.\d+)', str(Rebounds))

Rebounds_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Rebounds))

# NBA Assists
with open(save_path+'draftkings_NBA_Assists.txt', 'r') as file:
    Assists = file.read()
Assists = Assists.replace("âˆ’", "-")

Assists_Line = re.findall('line">(\d+.\d+)', str(Assists))

Assists_Odds = re.findall('color">(.\d+)', str(Assists))

Assists_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(Assists))

# NBA Threes
with open(save_path+'draftkings_NBA_Threes.txt', 'r') as file:
    THREES = file.read()
THREES = THREES.replace("âˆ’", "-")

THREES_Line = re.findall('line">(\d+.\d+)', str(THREES))

THREES_Odds = re.findall('color">(.\d+)', str(THREES))

THREES_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(THREES))

# NBA Steals
with open(save_path+'draftkings_NBA_STEALS.txt', 'r') as file:
    STEALS = file.read()
STEALS = STEALS.replace("âˆ’", "-")

STEALS_Line = re.findall('line">(\d+.\d+)', str(STEALS))

STEALS_Odds = re.findall('color">(.\d+)', str(STEALS))

STEALS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(STEALS))

# NBA Turnovers
with open(save_path+'draftkings_NBA_TO.txt', 'r') as file:
    TO = file.read()
TO = TO.replace("âˆ’", "-")

TO_Line = re.findall('line">(\d+.\d+)', str(TO))

TO_Odds = re.findall('color">(.\d+)', str(TO))

TO_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(TO))

In [48]:
Upd_Odds13 = []
Upd_Lines13 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in NBA_PTS_Line:
    Upd_Lines13.append(NBA_PTS_Line[First:Second])
    First += 2
    Second += 2

for i in NBA_PTS_Odds:
    Upd_Odds13.append(NBA_PTS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


NBA_PTS_Odds_DF = pd.DataFrame(Upd_Odds13)
NBA_PTS_Odds_DF = NBA_PTS_Odds_DF.dropna()

NBA_PTS_Lines_DF = pd.DataFrame(Upd_Lines13)
NBA_PTS_Lines_DF = NBA_PTS_Lines_DF.dropna()
NBA_PTS_Lines_DF = NBA_PTS_Lines_DF.drop([0],axis=1)

In [49]:
Upd_Odds14 = []
Upd_Lines14 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Rebounds_Line:
    Upd_Lines14.append(Rebounds_Line[First:Second])
    First += 2
    Second += 2

for i in Rebounds_Odds:
    Upd_Odds14.append(Rebounds_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


Rebounds_Odds_DF = pd.DataFrame(Upd_Odds14)
Rebounds_Odds_DF = Rebounds_Odds_DF.dropna()

Rebounds_Lines_DF = pd.DataFrame(Upd_Lines14)
Rebounds_Lines_DF = Rebounds_Lines_DF.dropna()
Rebounds_Lines_DF = Rebounds_Lines_DF.drop([0],axis=1)

In [50]:
Upd_Odds16 = []
Upd_Lines16 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in Assists_Line:
    Upd_Lines16.append(Assists_Line[First:Second])
    First += 2
    Second += 2

for i in Assists_Odds:
    Upd_Odds16.append(Assists_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


Assists_Odds_DF = pd.DataFrame(Upd_Odds16)
Assists_Odds_DF = Assists_Odds_DF.dropna()

Assists_Lines_DF = pd.DataFrame(Upd_Lines16)
Assists_Lines_DF = Assists_Lines_DF.dropna()
Assists_Lines_DF = Assists_Lines_DF.drop([0],axis=1)

In [51]:
Upd_Odds17 = []
Upd_Lines17 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in THREES_Line:
    Upd_Lines17.append(THREES_Line[First:Second])
    First += 2
    Second += 2

for i in THREES_Odds:
    Upd_Odds17.append(THREES_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


THREES_Odds_DF = pd.DataFrame(Upd_Odds17)
THREES_Odds_DF = THREES_Odds_DF.dropna()

THREES_Lines_DF = pd.DataFrame(Upd_Lines17)
THREES_Lines_DF = THREES_Lines_DF.dropna()
THREES_Lines_DF = THREES_Lines_DF.drop([0],axis=1)

In [52]:
Upd_Odds19 = []
Upd_Lines19 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in STEALS_Line:
    Upd_Lines19.append(STEALS_Line[First:Second])
    First += 2
    Second += 2

for i in STEALS_Odds:
    Upd_Odds19.append(STEALS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


STEALS_Odds_DF = pd.DataFrame(Upd_Odds19)
STEALS_Odds_DF = STEALS_Odds_DF.dropna()

STEALS_Lines_DF = pd.DataFrame(Upd_Lines19)
STEALS_Lines_DF = STEALS_Lines_DF.dropna()
STEALS_Lines_DF = STEALS_Lines_DF.drop([0],axis=1)

In [53]:
Upd_Odds22 = []
Upd_Lines22 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in TO_Line:
    Upd_Lines22.append(TO_Line[First:Second])
    First += 2
    Second += 2

for i in TO_Odds:
    Upd_Odds22.append(TO_Odds[First2:Second2])
    First2 += 2
    Second2 += 2

TO_Odds_DF = pd.DataFrame(Upd_Odds22)
TO_Odds_DF = TO_Odds_DF.dropna()

TO_Lines_DF = pd.DataFrame(Upd_Lines22)
TO_Lines_DF = TO_Lines_DF.dropna()
TO_Lines_DF = TO_Lines_DF.drop([0],axis=1)


#### Create dataframes for Draftkings data

In [54]:
NBA_PTS_DK = pd.DataFrame(list(zip(NBA_PTS_Players)),
               columns =['Name'])

NBA_PTS_DK = NBA_PTS_DK.join(NBA_PTS_Lines_DF)
NBA_PTS_DK = NBA_PTS_DK.join(NBA_PTS_Odds_DF, lsuffix="fa", rsuffix="werf")

NBA_PTS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

NBA_PTS_DK[['O Odds', 'Line', 'U Odds']] = NBA_PTS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [55]:
Rebounds_DK = pd.DataFrame(list(zip(Rebounds_Players)),
               columns =['Name'])

Rebounds_DK = Rebounds_DK.join(Rebounds_Lines_DF)
Rebounds_DK = Rebounds_DK.join(Rebounds_Odds_DF, lsuffix="fa", rsuffix="werf")

Rebounds_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

Rebounds_DK[['O Odds', 'Line', 'U Odds']] = Rebounds_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [56]:
Assists_DK = pd.DataFrame(list(zip(Assists_Players)),
               columns =['Name'])

Assists_DK = Assists_DK.join(Assists_Lines_DF)
Assists_DK = Assists_DK.join(Assists_Odds_DF, lsuffix="fa", rsuffix="werf")

Assists_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

Assists_DK[['O Odds', 'Line', 'U Odds']] = Assists_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [57]:
THREES_DK = pd.DataFrame(list(zip(THREES_Players)),
               columns =['Name'])

THREES_DK = THREES_DK.join(THREES_Lines_DF)
THREES_DK = THREES_DK.join(THREES_Odds_DF, lsuffix="fa", rsuffix="werf")

THREES_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

THREES_DK[['O Odds', 'Line', 'U Odds']] = THREES_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [58]:
STEALS_DK = pd.DataFrame(list(zip(STEALS_Players)),
               columns =['Name'])

STEALS_DK = STEALS_DK.join(STEALS_Lines_DF)
STEALS_DK = STEALS_DK.join(STEALS_Odds_DF, lsuffix="fa", rsuffix="werf")

STEALS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

STEALS_DK[['O Odds', 'Line', 'U Odds']] = STEALS_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

In [59]:
TO_DK = pd.DataFrame(list(zip(TO_Players)),
               columns =['Name'])

TO_DK = TO_DK.join(TO_Lines_DF)
TO_DK = TO_DK.join(TO_Odds_DF, lsuffix="fa", rsuffix="werf")

TO_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

TO_DK[['O Odds', 'Line', 'U Odds']] = TO_DK[['O Odds', 'Line', 'U Odds']].apply(pd.to_numeric)

#### Format dataframes with probability formula

In [60]:
NBA_PTS_DK['O Prob'] = (NBA_PTS_DK['O Odds'].abs()/(NBA_PTS_DK['O Odds'].abs()+100)*100).round(1).where(NBA_PTS_DK['O Odds'] < 0, 100/(NBA_PTS_DK['O Odds']+100)*100).round(1)
NBA_PTS_DK['U Prob'] = (NBA_PTS_DK['U Odds'].abs()/(NBA_PTS_DK['U Odds'].abs()+100)*100).round(1).where(NBA_PTS_DK['U Odds'] < 0, 100/(NBA_PTS_DK['U Odds']+100)*100).round(1)
NBA_PTS_DK['Favor'] = np.where(NBA_PTS_DK['O Prob']> NBA_PTS_DK['U Prob'], "Over", "Under")

NBA_PTS_DK = NBA_PTS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
NBA_PTS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,8.5,47.6,59.2,Under
1,Aleksej Pokusevski,8.5,44.4,63.0,Under
2,Christian Wood,10.5,54.5,52.4,Over
3,Dorian Finney,11.5,63.0,43.5,Over
4,Dwight Powell,14.5,63.0,43.5,Over


In [61]:
Rebounds_DK['O Prob'] = (Rebounds_DK['O Odds'].abs()/(Rebounds_DK['O Odds'].abs()+100)*100).round(1).where(Rebounds_DK['O Odds'] < 0, 100/(Rebounds_DK['O Odds']+100)*100).round(1)
Rebounds_DK['U Prob'] = (Rebounds_DK['U Odds'].abs()/(Rebounds_DK['U Odds'].abs()+100)*100).round(1).where(Rebounds_DK['U Odds'] < 0, 100/(Rebounds_DK['U Odds']+100)*100).round(1)
Rebounds_DK['Favor'] = np.where(Rebounds_DK['O Prob']> Rebounds_DK['U Prob'], "Over", "Under")

Rebounds_DK = Rebounds_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
Rebounds_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,2.5,61.5,45.5,Over
1,Aleksej Pokusevski,4.5,51.2,56.5,Under
2,Christian Wood,7.5,45.5,61.5,Under
3,Dorian Finney,3.5,63.0,44.4,Over
4,Dwight Powell,2.5,38.5,68.3,Under


In [62]:
Assists_DK['O Prob'] = (Assists_DK['O Odds'].abs()/(Assists_DK['O Odds'].abs()+100)*100).round(1).where(Assists_DK['O Odds'] < 0, 100/(Assists_DK['O Odds']+100)*100).round(1)
Assists_DK['U Prob'] = (Assists_DK['U Odds'].abs()/(Assists_DK['U Odds'].abs()+100)*100).round(1).where(Assists_DK['U Odds'] < 0, 100/(Assists_DK['U Odds']+100)*100).round(1)
Assists_DK['Favor'] = np.where(Assists_DK['O Prob']> Assists_DK['U Prob'], "Over", "Under")

Assists_DK = Assists_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
Assists_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,2.5,55.6,51.2,Over
1,Aleksej Pokusevski,4.5,36.4,70.6,Under
2,Christian Wood,0.5,62.3,44.4,Over
3,Dorian Finney,1.5,57.4,48.8,Over
4,Luguentz Dort,0.5,56.5,51.2,Over


In [63]:
THREES_DK['O Prob'] = (THREES_DK['O Odds'].abs()/(THREES_DK['O Odds'].abs()+100)*100).round(1).where(THREES_DK['O Odds'] < 0, 100/(THREES_DK['O Odds']+100)*100).round(1)
THREES_DK['U Prob'] = (THREES_DK['U Odds'].abs()/(THREES_DK['U Odds'].abs()+100)*100).round(1).where(THREES_DK['U Odds'] < 0, 100/(THREES_DK['U Odds']+100)*100).round(1)
THREES_DK['Favor'] = np.where(THREES_DK['O Prob']> THREES_DK['U Prob'], "Over", "Under")

THREES_DK = THREES_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
THREES_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,0.5,36.4,71.0,Under
1,Aleksej Pokusevski,0.5,43.5,63.6,Under
2,Christian Wood,0.5,61.5,45.5,Over
3,Dorian Finney,2.5,63.6,43.5,Over
4,Luguentz Dort,0.5,61.5,45.5,Over


In [64]:
STEALS_DK['O Prob'] = (STEALS_DK['O Odds'].abs()/(STEALS_DK['O Odds'].abs()+100)*100).round(1).where(STEALS_DK['O Odds'] < 0, 100/(STEALS_DK['O Odds']+100)*100).round(1)
STEALS_DK['U Prob'] = (STEALS_DK['U Odds'].abs()/(STEALS_DK['U Odds'].abs()+100)*100).round(1).where(STEALS_DK['U Odds'] < 0, 100/(STEALS_DK['U Odds']+100)*100).round(1)
STEALS_DK['Favor'] = np.where(STEALS_DK['O Prob']> STEALS_DK['U Prob'], "Over", "Under")

STEALS_DK = STEALS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
STEALS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,0.5,30.3,76.7,Under
1,Aleksej Pokusevski,0.5,33.9,73.0,Under
2,Christian Wood,0.5,30.3,76.7,Under
3,Dorian Finney,0.5,37.0,70.1,Under
4,Dwight Powell,0.5,12.5,94.1,Under


In [65]:
TO_DK['O Prob'] = (TO_DK['O Odds'].abs()/(TO_DK['O Odds'].abs()+100)*100).round(1).where(TO_DK['O Odds'] < 0, 100/(TO_DK['O Odds']+100)*100).round(1)
TO_DK['U Prob'] = (TO_DK['U Odds'].abs()/(TO_DK['U Odds'].abs()+100)*100).round(1).where(TO_DK['U Odds'] < 0, 100/(TO_DK['U Odds']+100)*100).round(1)
TO_DK['Favor'] = np.where(TO_DK['O Prob']> TO_DK['U Prob'], "Over", "Under")

TO_DK = TO_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
TO_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Aaron Wiggins,0.5,44.4,62.3,Under
1,Aleksej Pokusevski,1.5,59.2,47.6,Over
2,Christian Wood,3.5,60.8,46.5,Over
3,Dorian Finney,0.5,34.5,72.2,Under
4,Dwight Powell,0.5,20.4,86.7,Under


#### Match lines from Draftkings to Prizepicks

In [66]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


NBA_PTS_TABLE = fuzzy_merge(PRIZEPICKS_table, NBA_PTS_DK, 'Player', 'Player', threshold=95)

NBA_PTS_TABLE = NBA_PTS_TABLE[NBA_PTS_TABLE['Stat Type'].str.contains('Points')]
NBA_PTS_TABLE = NBA_PTS_TABLE[NBA_PTS_TABLE['Sport'].str.endswith('NBA', na=False)]
NBA_PTS_TABLE['Player'] = NBA_PTS_TABLE['matches']

NBA_PTS_TABLE = NBA_PTS_TABLE.merge(NBA_PTS_DK, on='Player')

NBA_PTS_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [67]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


Rebounds_TABLE = fuzzy_merge(PRIZEPICKS_table, Rebounds_DK, 'Player', 'Player', threshold=95)

Rebounds_TABLE = Rebounds_TABLE[Rebounds_TABLE['Stat Type'].str.contains('Rebounds')]
Rebounds_TABLE = Rebounds_TABLE[Rebounds_TABLE['Sport'].str.endswith('NBA')]
Rebounds_TABLE['Player'] = Rebounds_TABLE['matches']

Rebounds_TABLE = Rebounds_TABLE.merge(Rebounds_DK, on='Player')

Rebounds_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [68]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


Assists_TABLE = fuzzy_merge(PRIZEPICKS_table, Assists_DK, 'Player', 'Player', threshold=95)

Assists_TABLE = Assists_TABLE[Assists_TABLE['Stat Type'].str.contains('Assists')]
Assists_TABLE = Assists_TABLE[Assists_TABLE['Sport'].str.endswith('NBA')]
Assists_TABLE['Player'] = Assists_TABLE['matches']

Assists_TABLE = Assists_TABLE.merge(Assists_DK, on='Player')

Assists_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [69]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


THREES_TABLE = fuzzy_merge(PRIZEPICKS_table, THREES_DK, 'Player', 'Player', threshold=95)

THREES_TABLE = THREES_TABLE[THREES_TABLE['Stat Type'].str.contains('3')]
THREES_TABLE = THREES_TABLE[THREES_TABLE['Sport'].str.endswith('NBA', na=False)]
THREES_TABLE['Player'] = THREES_TABLE['matches']

THREES_TABLE = THREES_TABLE.merge(THREES_DK, on='Player')

THREES_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [70]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


STEALS_TABLE = fuzzy_merge(PRIZEPICKS_table, STEALS_DK, 'Player', 'Player', threshold=95)

STEALS_TABLE = STEALS_TABLE[STEALS_TABLE['Stat Type'].str.contains('Steal')]
STEALS_TABLE = STEALS_TABLE[STEALS_TABLE['Sport'].str.endswith('NBA', na=False)]
STEALS_TABLE['Player'] = STEALS_TABLE['matches']

STEALS_TABLE = STEALS_TABLE.merge(STEALS_DK, on='Player')

STEALS_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [71]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


TO_TABLE = fuzzy_merge(PRIZEPICKS_table, TO_DK, 'Player', 'Player', threshold=95)

TO_TABLE = TO_TABLE[TO_TABLE['Stat Type'].str.contains('Turnover')]
TO_TABLE = TO_TABLE[TO_TABLE['Sport'].str.endswith('NBA', na=False)]
TO_TABLE['Player'] = TO_TABLE['matches']

TO_TABLE = TO_TABLE.merge(TO_DK, on='Player')

TO_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


## NHL

#### Clean the NBA data from Draftkings

In [72]:
# NHL Shots On Goal
with open(save_path+'draftkings_NHL_SOG.txt', 'r') as file:
    NHL_SOG = file.read()
NHL_SOG = NHL_SOG.replace("âˆ’", "-")

NHL_SOG_Line = re.findall('line">(\d+.\d+)', str(NHL_SOG))

NHL_SOG_Odds = re.findall('color">(.\d+)', str(NHL_SOG))

NHL_SOG_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?)', str(NHL_SOG))

# NHL Points
with open(save_path+'draftkings_NHL_PTS.txt', 'r') as file:
    NHL_POINTS = file.read()
NHL_POINTS = NHL_POINTS.replace("âˆ’", "-")

NHL_POINTS_Line = re.findall('line">(\d+.\d+)', str(NHL_POINTS))

NHL_POINTS_Odds = re.findall('color">(.\d+)', str(NHL_POINTS))

NHL_POINTS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?[ .-]?\w+)', str(NHL_POINTS))

In [73]:
Upd_Odds10 = []
Upd_Lines10 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in NHL_SOG_Line:
    Upd_Lines10.append(NHL_SOG_Line[First:Second])
    First += 2
    Second += 2

for i in NHL_SOG_Odds:
    Upd_Odds10.append(NHL_SOG_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


NHL_SOG_Odds_DF = pd.DataFrame(Upd_Odds10)
NHL_SOG_Odds_DF = NHL_SOG_Odds_DF.dropna()

NHL_SOG_Lines_DF = pd.DataFrame(Upd_Lines10)
NHL_SOG_Lines_DF = NHL_SOG_Lines_DF.drop([0],axis=1)

In [74]:
Upd_Odds11 = []
Upd_Lines11 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in NHL_POINTS_Line:
    Upd_Lines11.append(NHL_POINTS_Line[First:Second])
    First += 2
    Second += 2

for i in NHL_POINTS_Odds:
    Upd_Odds11.append(NHL_POINTS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


NHL_POINTS_Odds_DF = pd.DataFrame(Upd_Odds11)
NHL_POINTS_Odds_DF = NHL_POINTS_Odds_DF.dropna()

NHL_POINTS_Lines_DF = pd.DataFrame(Upd_Lines11)
NHL_POINTS_Lines_DF = NHL_POINTS_Lines_DF.drop([0],axis=1)

#### Create dataframes for Draftkings data

In [75]:
NHL_SOG_DK = pd.DataFrame(list(zip(NHL_SOG_Players)),
               columns =['Name'])

NHL_SOG_DK = NHL_SOG_DK.join(NHL_SOG_Lines_DF)
NHL_SOG_DK = NHL_SOG_DK.join(NHL_SOG_Odds_DF, lsuffix="fa", rsuffix="werf")

NHL_SOG_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

NHL_SOG_DK['O Odds'] = pd.to_numeric(NHL_SOG_DK['O Odds'])
NHL_SOG_DK['U Odds'] = pd.to_numeric(NHL_SOG_DK['U Odds'])
NHL_SOG_DK['Line'] = pd.to_numeric(NHL_SOG_DK['Line'])

In [76]:
NHL_POINTS_DK = pd.DataFrame(list(zip(NHL_POINTS_Players)),
               columns =['Name'])

NHL_POINTS_DK = NHL_POINTS_DK.join(NHL_POINTS_Lines_DF)
NHL_POINTS_DK = NHL_POINTS_DK.join(NHL_POINTS_Odds_DF, lsuffix="fa", rsuffix="werf")

NHL_POINTS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

NHL_POINTS_DK['O Odds'] = pd.to_numeric(NHL_POINTS_DK['O Odds'])
NHL_POINTS_DK['U Odds'] = pd.to_numeric(NHL_POINTS_DK['U Odds'])
NHL_POINTS_DK['Line'] = pd.to_numeric(NHL_POINTS_DK['Line'])

#### Format dataframes with probability formula

In [77]:
NHL_SOG_DK['O Prob'] = (NHL_SOG_DK['O Odds'].abs()/(NHL_SOG_DK['O Odds'].abs()+100)*100).round(1).where(NHL_SOG_DK['O Odds'] < 0, 100/(NHL_SOG_DK['O Odds']+100)*100).round(1)
NHL_SOG_DK['U Prob'] = (NHL_SOG_DK['U Odds'].abs()/(NHL_SOG_DK['U Odds'].abs()+100)*100).round(1).where(NHL_SOG_DK['U Odds'] < 0, 100/(NHL_SOG_DK['U Odds']+100)*100).round(1)
NHL_SOG_DK['Favor'] = np.where(NHL_SOG_DK['O Prob']> NHL_SOG_DK['U Prob'], "Over", "Under")

NHL_SOG_DK = NHL_SOG_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
NHL_SOG_DK.head()

,Player,Line,O Prob,U Prob,Favor


In [78]:
NHL_POINTS_DK['O Prob'] = (NHL_POINTS_DK['O Odds'].abs()/(NHL_POINTS_DK['O Odds'].abs()+100)*100).round(1).where(NHL_POINTS_DK['O Odds'] < 0, 100/(NHL_POINTS_DK['O Odds']+100)*100).round(1)
NHL_POINTS_DK['U Prob'] = (NHL_POINTS_DK['U Odds'].abs()/(NHL_POINTS_DK['U Odds'].abs()+100)*100).round(1).where(NHL_POINTS_DK['U Odds'] < 0, 100/(NHL_POINTS_DK['U Odds']+100)*100).round(1)
NHL_POINTS_DK['Favor'] = np.where(NHL_POINTS_DK['O Prob']> NHL_POINTS_DK['U Prob'], "Over", "Under")

NHL_POINTS_DK = NHL_POINTS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
NHL_POINTS_DK.head()

,Player,Line,O Prob,U Prob,Favor


#### Match lines from Draftkings to Prizepicks

In [79]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


NHL_SOG_TABLE = fuzzy_merge(PRIZEPICKS_table, NHL_SOG_DK, 'Player', 'Player', threshold=95)

NHL_SOG_TABLE = NHL_SOG_TABLE[NHL_SOG_TABLE['Stat Type'].str.contains('Shots On Goal')]
NHL_SOG_TABLE = NHL_SOG_TABLE[NHL_SOG_TABLE['Sport'].str.contains('NHL')]
NHL_SOG_TABLE['Player'] = NHL_SOG_TABLE['matches']

NHL_SOG_TABLE = NHL_SOG_TABLE.merge(NHL_SOG_DK, on='Player')

NHL_SOG_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


In [80]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


NHL_POINTS_TABLE = fuzzy_merge(PRIZEPICKS_table, NHL_POINTS_DK, 'Player', 'Player', threshold=95)

NHL_POINTS_TABLE = NHL_POINTS_TABLE[NHL_POINTS_TABLE['Stat Type'].str.contains('Points')]
NHL_POINTS_TABLE = NHL_POINTS_TABLE[NHL_POINTS_TABLE['Sport'].str.contains('NHL', na=False)]
NHL_POINTS_TABLE['Player'] = NHL_POINTS_TABLE['matches']

NHL_POINTS_TABLE = NHL_POINTS_TABLE.merge(NHL_POINTS_DK, on='Player')

NHL_POINTS_TABLE.head()

,Team,Sport,Stat Type,Line_x,Last Updated,matches,Player,Line_y,O Prob,U Prob,Favor


## CFB

#### Clean the NBA data from Draftkings

In [81]:
# CFB Passing Yards
with open(save_path+'draftkings_CFB_PassYds.txt', 'r') as file:
    CFB_PASSYDS = file.read()
CFB_PASSYDS = CFB_PASSYDS.replace("âˆ’", "-")

CFB_PASSYDS_Line = re.findall('line">(\d+.\d+)', str(CFB_PASSYDS))

CFB_PASSYDS_Odds = re.findall('color">(.\d+)', str(CFB_PASSYDS))

CFB_PASSYDS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?[ .-]?\w+)', str(CFB_PASSYDS))

# CFB Passing Yards
with open(save_path+'draftkings_CFB_PassTds.txt', 'r') as file:
    CFB_PASSTDS = file.read()
CFB_PASSTDS = CFB_PASSTDS.replace("âˆ’", "-")

CFB_PASSTDS_Line = re.findall('line">(\d+.\d+)', str(CFB_PASSTDS))

CFB_PASSTDS_Odds = re.findall('color">(.\d+)', str(CFB_PASSTDS))

CFB_PASSTDS_Players = re.findall('row-name">(\w+.[ .-]?\w+\s?[\w+]?[ .-]?\w+)', str(CFB_PASSTDS))



In [82]:
Upd_Odds30 = []
Upd_Lines30 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in CFB_PASSYDS_Line:
    Upd_Lines30.append(CFB_PASSYDS_Line[First:Second])
    First += 2
    Second += 2

for i in CFB_PASSYDS_Odds:
    Upd_Odds30.append(CFB_PASSYDS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


CFB_PASSYDS_Odds_DF = pd.DataFrame(Upd_Odds30)
CFB_PASSYDS_Odds_DF = CFB_PASSYDS_Odds_DF.dropna()

CFB_PASSYDS_Lines_DF = pd.DataFrame(Upd_Lines30)
CFB_PASSYDS_Lines_DF = CFB_PASSYDS_Lines_DF.drop([0],axis=1)

In [85]:
Upd_Odds31 = []
Upd_Lines31 = []
First = 0
Second = 2
First2 = 0
Second2 = 2

for i in CFB_PASSTDS_Line:
    Upd_Lines31.append(CFB_PASSTDS_Line[First:Second])
    First += 2
    Second += 2

for i in CFB_PASSTDS_Odds:
    Upd_Odds31.append(CFB_PASSTDS_Odds[First2:Second2])
    First2 += 2
    Second2 += 2


CFB_PASSTDS_Odds_DF = pd.DataFrame(Upd_Odds31)
CFB_PASSTDS_Odds_DF = CFB_PASSTDS_Odds_DF.dropna()

CFB_PASSTDS_Lines_DF = pd.DataFrame(Upd_Lines31)
CFB_PASSTDS_Lines_DF = CFB_PASSTDS_Lines_DF.drop([0],axis=1)

#### Create dataframes for Draftkings data

In [86]:
CFB_PASSYDS_DK = pd.DataFrame(list(zip(CFB_PASSYDS_Players)),
               columns =['Name'])

CFB_PASSYDS_DK = CFB_PASSYDS_DK.join(CFB_PASSYDS_Lines_DF)
CFB_PASSYDS_DK = CFB_PASSYDS_DK.join(CFB_PASSYDS_Odds_DF, lsuffix="fa", rsuffix="werf")

CFB_PASSYDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

CFB_PASSYDS_DK['O Odds'] = pd.to_numeric(CFB_PASSYDS_DK['O Odds'])
CFB_PASSYDS_DK['U Odds'] = pd.to_numeric(CFB_PASSYDS_DK['U Odds'])
CFB_PASSYDS_DK['Line'] = pd.to_numeric(CFB_PASSYDS_DK['Line'])

In [87]:
CFB_PASSTDS_DK = pd.DataFrame(list(zip(CFB_PASSTDS_Players)),
               columns =['Name'])

CFB_PASSTDS_DK = CFB_PASSTDS_DK.join(CFB_PASSTDS_Lines_DF)
CFB_PASSTDS_DK = CFB_PASSTDS_DK.join(CFB_PASSTDS_Odds_DF, lsuffix="fa", rsuffix="werf")

CFB_PASSTDS_DK.columns = ["Player", "Line", "O Odds", "U Odds"]

CFB_PASSTDS_DK['O Odds'] = pd.to_numeric(CFB_PASSTDS_DK['O Odds'])
CFB_PASSTDS_DK['U Odds'] = pd.to_numeric(CFB_PASSTDS_DK['U Odds'])
CFB_PASSTDS_DK['Line'] = pd.to_numeric(CFB_PASSTDS_DK['Line'])

#### Format dataframes with probability formula

In [88]:
CFB_PASSYDS_DK['O Prob'] = (CFB_PASSYDS_DK['O Odds'].abs()/(CFB_PASSYDS_DK['O Odds'].abs()+100)*100).round(1).where(CFB_PASSYDS_DK['O Odds'] < 0, 100/(CFB_PASSYDS_DK['O Odds']+100)*100).round(1)
CFB_PASSYDS_DK['U Prob'] = (CFB_PASSYDS_DK['U Odds'].abs()/(CFB_PASSYDS_DK['U Odds'].abs()+100)*100).round(1).where(CFB_PASSYDS_DK['U Odds'] < 0, 100/(CFB_PASSYDS_DK['U Odds']+100)*100).round(1)
CFB_PASSYDS_DK['Favor'] = np.where(CFB_PASSYDS_DK['O Prob']> CFB_PASSYDS_DK['U Prob'], "Over", "Under")

CFB_PASSYDS_DK = CFB_PASSYDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
CFB_PASSYDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Dorian Thompson-Robinson,269.5,53.5,53.5,Under
1,Tanner McKee,270.5,53.5,53.5,Under


In [89]:
CFB_PASSTDS_DK['O Prob'] = (CFB_PASSTDS_DK['O Odds'].abs()/(CFB_PASSTDS_DK['O Odds'].abs()+100)*100).round(1).where(CFB_PASSTDS_DK['O Odds'] < 0, 100/(CFB_PASSTDS_DK['O Odds']+100)*100).round(1)
CFB_PASSTDS_DK['U Prob'] = (CFB_PASSTDS_DK['U Odds'].abs()/(CFB_PASSTDS_DK['U Odds'].abs()+100)*100).round(1).where(CFB_PASSTDS_DK['U Odds'] < 0, 100/(CFB_PASSTDS_DK['U Odds']+100)*100).round(1)
CFB_PASSTDS_DK['Favor'] = np.where(CFB_PASSTDS_DK['O Prob']> CFB_PASSTDS_DK['U Prob'], "Over", "Under")

CFB_PASSTDS_DK = CFB_PASSTDS_DK.drop(columns=['O Odds', 'U Odds'], axis=1)
CFB_PASSTDS_DK.head()

,Player,Line,O Prob,U Prob,Favor
0,Dorian Thompson-Robinson,2.5,47.6,59.2,Under
1,Tanner McKee,1.5,52.4,54.5,Under


#### Match lines from Draftkings to Prizepicks

In [90]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


CFB_PASSYDS_TABLE = fuzzy_merge(PRIZEPICKS_table, CFB_PASSYDS_DK, 'Player', 'Player', threshold=95)

CFB_PASSYDS_TABLE = CFB_PASSYDS_TABLE[CFB_PASSYDS_TABLE['Stat Type'].str.contains('Pass Yard')]
CFB_PASSYDS_TABLE = CFB_PASSYDS_TABLE[CFB_PASSYDS_TABLE['Sport'].str.contains('CFB', na=False)]
CFB_PASSYDS_TABLE['Player'] = CFB_PASSYDS_TABLE['matches']

CFB_PASSYDS_TABLE = CFB_PASSYDS_TABLE.merge(CFB_PASSYDS_DK, on='Player')
CFB_PASSYDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Dorian Thompson-Robinson,UCLA,CFB,Pass Yards,265.5,10/27/22 12:50 PM,Dorian Thompson-Robinson,269.5,53.5,53.5,Under
1,Tanner McKee,STA,CFB,Pass Yards,270.5,10/28/22 01:27 PM,Tanner McKee,270.5,53.5,53.5,Under


In [91]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1


CFB_PASSTDS_TABLE = fuzzy_merge(PRIZEPICKS_table, CFB_PASSTDS_DK, 'Player', 'Player', threshold=95)

CFB_PASSTDS_TABLE = CFB_PASSTDS_TABLE[CFB_PASSTDS_TABLE['Stat Type'].str.contains('Pass TD')]
CFB_PASSTDS_TABLE = CFB_PASSTDS_TABLE[CFB_PASSTDS_TABLE['Sport'].str.contains('CFB', na=False)]
CFB_PASSTDS_TABLE['Player'] = CFB_PASSTDS_TABLE['matches']

CFB_PASSTDS_TABLE = CFB_PASSTDS_TABLE.merge(CFB_PASSTDS_DK, on='Player')
CFB_PASSTDS_TABLE.head()

,Player,Team,Sport,Stat Type,Line_x,Last Updated,matches,Line_y,O Prob,U Prob,Favor
0,Dorian Thompson-Robinson,UCLA,CFB,Pass TDs,2.0,10/27/22 12:53 PM,Dorian Thompson-Robinson,2.5,47.6,59.2,Under
1,Tanner McKee,STA,CFB,Pass TDs,1.5,10/28/22 03:26 PM,Tanner McKee,1.5,52.4,54.5,Under


## Main table

#### Format Final (Main) Table

In [92]:
MAIN_TABLE = pd.concat([
    HITSALLOWED_TABLE, STRIKEOUTS_TABLE, RUNS_TABLE,
    RUSHYDS_TABLE, PASSTDS_TABLE, PASSYDS_TABLE, RUSHATT_TABLE, REC_TABLE, RECYDS_TABLE, PassAtt_TABLE,
    NBA_PTS_TABLE, Rebounds_TABLE, Assists_TABLE, THREES_TABLE, STEALS_TABLE, TO_TABLE,
    NHL_POINTS_TABLE, NHL_SOG_TABLE,
    CFB_PASSYDS_TABLE, PASSTDS_TABLE])


MAIN_TABLE['Favor'] = np.where((MAIN_TABLE['Line_y'] > MAIN_TABLE['Line_x']) & (MAIN_TABLE['Favor'] == 'Over'), MAIN_TABLE['Favor']+" +", MAIN_TABLE['Favor'])
MAIN_TABLE['Favor'] = np.where((MAIN_TABLE['Line_y'] < MAIN_TABLE['Line_x']) & (MAIN_TABLE['Favor'] == 'Under'), MAIN_TABLE['Favor']+" +", MAIN_TABLE['Favor'])  
MAIN_TABLE['Line_y'] = np.where((MAIN_TABLE['Line_x']!= MAIN_TABLE['Line_y']) & (MAIN_TABLE['Favor'].str.contains(" +") == False), pd.NaT, "Under")
MAIN_TABLE['Player'].replace('', np.nan, inplace=True)
MAIN_TABLE = MAIN_TABLE.dropna()
MAIN_TABLE['Probability (%)'] = np.where(MAIN_TABLE['O Prob']== MAIN_TABLE['U Prob'], 50, MAIN_TABLE[["O Prob", "U Prob"]].max(axis=1))
MAIN_TABLE = MAIN_TABLE.drop(columns=['matches', 'Line_y', 'O Prob', 'U Prob'], axis=1)
MAIN_TABLE = MAIN_TABLE.rename(columns={'Line_x': 'Line'})
MAIN_TABLE = MAIN_TABLE[['Player', 'Team', 'Sport', 'Stat Type', 'Last Updated', 'Line', 'Probability (%)', 'Favor']]

MAIN_TABLE.to_csv("MAIN_TABLE.csv", index=False)
MAIN_TABLE


,Player,Team,Sport,Stat Type,Last Updated,Line,Probability (%),Favor
1,Melvin Gordon I,DEN,NFL,Rush Yards,10/27/22 01:12 PM,39.5,54.5,Under
3,Latavius Murray,DEN,NFL,Rush Yards,10/27/22 01:07 PM,37.5,50.0,Under +
4,Kyler Murray,ARI,NFL,Rush Yards,10/28/22 05:04 PM,32.5,50.0,Under
6,Eno Benjamin,ARI,NFL,Rush Yards,10/28/22 05:05 PM,50.5,50.0,Under
7,Dalvin Cook,MIN,NFL,Rush Yards,10/28/22 04:48 PM,77.5,50.0,Under +
...,...,...,...,...,...,...,...,...
0,Kyler Murray,ARI,NFL,Pass TDs,10/28/22 05:02 PM,1.5,58.3,Under
1,Dak Prescott,DAL,NFL,Pass TDs,10/27/22 01:34 AM,1.5,57.4,Over
2,Derek Carr,LV,NFL,Pass TDs,10/27/22 01:46 AM,1.5,57.4,Over
3,Andy Dalton,NO,NFL,Pass TDs,10/27/22 01:34 AM,1.5,57.4,Under


In [93]:
# Importing required library
import pygsheets
  
# Create the Client
# Enter the name of the downloaded KEYS 
# file in service_account_file
client = pygsheets.authorize(service_account_file="main-table-364421-340b276b1a8e.json")


sh = client.open('Main Table')
wks = sh[0]
wks.clear()
wks.set_dataframe(MAIN_TABLE,(1,1))